In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:

import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from numpy.random import seed
from sklearn.utils import class_weight

from inception_model_generator_BN import MRNet_inc_model

from vgg_model_generator import MRNet_vgg_model
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc


Using TensorFlow backend.


#trainning on augmented balanced data with only the skewed class augmented
20 epochs


In [0]:
label_type = 'meniscus'
exam_view = 'sagittal'
model, cbs = MRNet_inc_model(1,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
print(total)
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
print(total)
neg = total - pos
class_count_test = {0: neg, 1: pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)

model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN_smalllr/meniscus/axial/weights.50.hdf5')
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=100, initial_epoch = 50,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

Tensor("sequential_1/mr_net_inception_layer_1/dropout_1/cond/Identity:0", shape=(1, 2048), dtype=float32)
Tensor("sequential_1/mr_net_inception_layer_1_1/dropout_1/cond/Identity:0", shape=(1, 2048), dtype=float32)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 21691809  
Total params: 21,691,809
Trainable params: 21,622,945
Non-trainable params: 68,864
_________________________________________________________________
947
113
Initializing Data Generator:
factor :  1.721264367816092
model:  inception
data type:  train
Combination:  meniscus  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1198
input size:  (299, 299)
Initializing Data Generator:
factor :  2.6451612903225805
model:  inception
data type:  test
Combination:  meniscus  and  sagittal
data path:  /co

## training on augmneted data

In [0]:
label_type = 'meniscus'
exam_view = 'sagittal'
model, cbs = MRNet_inc_model(2,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
print(total)
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
print(total)
neg = total - pos
class_count_test = {0: neg, 1: pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=1)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=1)

# model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN/meniscus/sagittal/weights.20.hdf5')
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=20, initial_epoch = 4,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 21691809  
Total params: 21,691,809
Trainable params: 21,622,945
Non-trainable params: 68,864
_________________________________________________________________
113
113
Initializing Data Generator:
factor :  2.6451612903225805
model:  inception
data type:  test
Combination:  meniscus  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  163
input size:  (299, 299)
Initializing Data Generator:
factor :  2.6451612903225805
model:  inception
data type:  test
Combination:  meniscus  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  163
input size:  (299, 299)
Epoch 5/20


# testing on validation dataset

In [5]:
label_type = 'meniscus'
exam_view = 'sagittal'
model, cbs = MRNet_inc_model(1,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)


# model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN_dropout/meniscus/sagittal/weights.50.hdf5')

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])
print(total)
neg = total - pos
class_count_valid = {0: neg, 1: pos}

batch_size = 1
valid_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_valid, batch_size=1,model="inception",data_type='valid', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)

metrics = model.evaluate(x=valid_data_gen,verbose=1,steps=total,use_multiprocessing=True,workers=6);
model.metrics_names
print(metrics)

Tensor("sequential_1/mr_net_inception_layer_1/dropout_1/cond/Identity:0", shape=(1, 2048), dtype=float32)
Tensor("sequential_1/mr_net_inception_layer_1_1/dropout_1/cond/Identity:0", shape=(1, 2048), dtype=float32)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 21691809  
Total params: 21,691,809
Trainable params: 21,622,945
Non-trainable params: 68,864
_________________________________________________________________
120
Initializing Data Generator:
factor :  1.3076923076923077
model:  inception
data type:  valid
Combination:  meniscus  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (299, 299)
120/120 [==============================] - 941s 8s/step
[0.6937185525894165, 25.924999237060547, 34.57500076293945, 0.0, 0.0, 0.4325309991836548, 0.43

In [0]:
print(model.predict(x=valid_data_gen,verbose=1,steps=5,use_multiprocessing=True,workers=6))

In [0]:
label_type = 'meniscus'
exam_view = 'sagittal'
model, cbs =  MRNet_inc_model(1,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

# model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN_nontrainable/meniscus/sagittal/weights.21.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])
neg = total - pos
class_count_valid = {0: neg, 1: pos}
batch_size = 1
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , shuffle=0, batch_size=1,model="inception",data_type='valid', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
print(model.evaluate_generator(valid_data_gen, total, verbose=1))
print(model.metrics_names)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_9 (MR (None, 1)                 21691809  
Total params: 21,691,809
Trainable params: 21,622,945
Non-trainable params: 68,864
_________________________________________________________________
Initializing Data Generator:
factor :  1.3076923076923077
model:  inception
data type:  valid
Combination:  meniscus  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (299, 299)
120/120 [==============================] - 55s 454ms/step
[0.6943026781082153, 0.0, 0.0, 36.04999923706055, 24.450000762939453, 0.6142098307609558, 0.0, 0.0, 0.4958333373069763]
['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']


In [0]:
label_type = 'meniscus'
exam_view = 'sagittal'
model, cbs =  MRNet_inc_model(1,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN_dropout/"+label_type+"/"+exam_view+"/weights.50.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])
neg = total - pos
class_count_valid = {0: neg, 1: pos}
batch_size = 1
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , shuffle=1, batch_size=1,model="inception",data_type='valid', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
# print(model.predict(valid_data_gen, batch_size=None,verbose=1))
# print(model.metrics_names)
tp = 0
tn = 0
fp = 0
fn = 0
y_true = []
y_score = []
for i in range(len(valid_data_gen)):
  x, y = next(valid_data_gen)
  _y = model.predict_classes(x, batch_size=1, verbose=0)
  _y_score = model.predict_proba(x, batch_size=1)
  y_score.append(_y_score[0][0])
  y_true.append(y[0])
  if _y[0][0] == 1:
    if _y[0][0] == y[0]:
      tp += 1
    else:
      fp += 1
  else:
    if _y[0][0] == y[0]:
      tn += 1
    else:
      fn += 1
y_score = np.array(y_score)
y_true = np.array(y_true)
fpr, tpr, _ = roc_curve(y_true, y_score)
auc = roc_auc(fpr, tpr)
print ("tp = ", tp, " fp = ", fp, " tn = ", tn, " fn = ", fn, " accuracy = ", (tp+tn)/(len(valid_data_gen)), " auc = ", auc)

# model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=12, verbose=True, callbacks=cbs,shuffle=False, use_multiprocessing=True, workers=6)